<a href="https://colab.research.google.com/github/nafisenik/DataMinig_pySpark/blob/main/question_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Question 5

In [1]:
! pip install -q pyspark

In [2]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
! tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster("local").setAppName("DegreesOfSeparation")
sc = SparkContext(conf = conf)

### BFS Search

In [5]:
from collections import defaultdict
class Graph:

  def __init__(self):
    self.graph = defaultdict(list)

  def add_edge(self,u,v):
    self.graph[u].append(v)

  def BFS(self, n):
    visited_node =  [0 for i in range(len(self.graph))]
    queue = []
    queue.append(n)
    visited_node[n] = 1
    while queue:
      node = queue.pop(0)
      print (node, end = "--->")
      for n in self.graph[node]:
        if visited_node[n] == 0:
          queue.append(n)
          visited_node[n] = 1

### Test BFS search

In [6]:
g = Graph()
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 2)
g.add_edge(2, 0)
g.add_edge(2, 3)
g.add_edge(3, 3)

print ("Following is Breadth First Traversal"
				" (starting from vertex 2)")
g.BFS(2)

Following is Breadth First Traversal (starting from vertex 2)
2--->0--->3--->1--->

In [7]:
https://github.com/shakshimaheshwari/MostPopularSuperHeroAndDegreeofSeperation/tree/master/MostPopularSuperHero
startCharacterID = 5306 #SpiderMan
targetCharacterID = 14  #ADAM 3,031 (who?)
hitCounter = sc.accumulator(0)

def convertToBFS(line):
	fields = line.split()
	connections = []
	heroID = int(fields[0])
	for connection in fields[1:]:
		connections.append(int(connection))
	distance = 9999 # infinity
	color = "WHITE"
	if (heroID == startCharacterID):
		color = "GRAY"
		distance = 0
	return (heroID, (connections, distance, color))

def bfsMap(node):

	heroID = node[0]
	connections = node[1][0]
	distance = node[1][1]
	color = node[1][2]
	results = []
	if (color == "GRAY"):
		for connection in connections:
			newDistance = distance + 1
			if (connection == targetCharacterID):
				hitCounter.add(1)
				
			results.append( (connection, ([], newDistance, "GRAY") ) )
		color = "BLACK" # color 'Black' the processed node
	# Emit the input node so we don't lose it
	results.append( (heroID, (connections, distance, color)) )
	return results

def bfsReduce(data1, data2):
	connection1, connection2 = data1[0], data2[0]
	distance1, distance2 = data1[1], data2[1]
	color1, color2 = data1[2], data2[2]
	edges, distance, color = [], 9999, color1
	if (len(connection1) > 0):
		edges.extend(connection1) #use extend b/c append all elements within list

	if (len(connection2) > 0):
		edges.extend(connection2)

	# Preserve the min distance
	if (distance > distance1):
		distance = distance1

	if (distance > distance2):
		distance = distance2

	# Preserve the darkest color
	if ( (color1 == 'WHITE') and (color2 == 'GRAY' or color2 == 'BLACK')):
		color = color2

	if ( (color1 == 'GRAY') and (color2 == 'BLACK') ):
		color = color2

	if ( (color2 == 'WHITE') and (color1 == 'GRAY' or color1 == 'BLACK')):
		color = color1

	if ( (color2 == 'GRAY') and (color1 == 'BLACK') ):
		color = color1

	return (edges, distance, color)
graph = sc.textFile("Marvel-graph.txt")
rdd = graph.map(convertToBFS)
for iteration in range(10):
	print("Running BFS iteration# " + str(iteration+1))
	mapper = rdd.flatMap(bfsMap)
	print("Processing " + str(mapper.count()) + "values")
	if (hitCounter.value > 0):
		print("Hit the target character! From " + str(hitCounter.value)+ "different directions.")
		break
	rdd = mapper.reduceByKey(bfsReduce)

Running BFS iteration# 1
Processing 8330values
Running BFS iteration# 2
Processing 220615values
Hit the target character! From 1different directions.


In [8]:
def parseLine(text):
    fields = text.split()
    superheroId = int(fields[0])
    return (superheroId, len(fields)-1)


def parseNames(text):
    fields = text.split('\"')
    return (int(fields[0]), fields[1].encode("utf-8"))

input = sc.textFile("Marvel-graph.txt")
superHeroIds = input.map(parseLine)

names = sc.textFile("Marvel-names.txt")
namesRdd = names.map(parseNames)

totalCount = superHeroIds.reduceByKey(lambda x,y : x+y)
flippedCount = totalCount.map(lambda item: (item[1], item[0]))
mostPouplar= flippedCount.max()

mostPopularName = namesRdd.lookup(mostPouplar[1])[0]
print(str(mostPopularName) + " is most popular superhero , with " + str(mostPouplar[0]) + " co-apperances")

b'CAPTAIN AMERICA' is most popular superhero , with 1933 co-apperances
